In [3]:
from loss import *
import torch
from models import *
from src.dataload import *
from tqdm.notebook import tqdm
from metrics.metrics import *


### Teste Dataloader

In [ ]:

#dataloader UIEBUIEB
train_loader_UIEB, test_loader_UIEB = create_dataloader(dataset_name="UIEB", dataset_path="data")

#dataloader TURBID
train_loader_TURBID, test_loader_TURBID = create_dataloader(dataset_name="TURBID", dataset_path="data")

#dataloader LSUI
train_loader_LSUI, test_loader_LSUI = create_dataloader(dataset_name="LSUI", dataset_path="data")

### Teste modelos

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = torch.randn(1, 3, 256, 256).to(device)
modelos = load_models()
for model in modelos:
    model = model.to(device)
    if type(model(x)) is tuple:
        print(model(x)[0].shape, model(x)[1].shape, model(x)[2].shape)
    else:
        print(model(x).shape)

if type(model(x)) is tuple:
        print(model(x)[0].shape, model(x)[1].shape, model(x)[2].shape)
    else:
        print(model(x).shape)

torch.Size([1, 3, 256, 256])
torch.Size([1, 3, 256, 256])
torch.Size([1, 3, 256, 256]) torch.Size([1, 16]) torch.Size([1, 16])


# Problema Atual
### Notas sobre as Loss
A loss lch nao esta funcionando. Ainda nao sei pq nao esta gerando resultado na loss.

Serao reajustadas para o intervalo de zero a 1 : (Feito)
* Histogram
* darkchannel
* hsv

Falta fazer

* lch nao funciona (Falta fazer)
### Notas

* Maiores problemas no autoencoder e em seu funcionamento. A dimensao latente de 16 nao gera nada aparentemente. 
* Talvez alterar o intervalo numero dos canais de cor ajude neste caso. O intervalo entre zero e 1 fazer 1/loss -1.
    * As funcoes de perda caso seja colocadas entre o intervalo de 0 a 1 ainda se comportam gerando valores negativos. Talvez esta seja uma caracteristica desse tip de funcao e do tipo de dado que esta sendo passado. Talvez passando os dados entre -1 e 1 isso possa ser ajustado. Essa transformacao serveria apenas para as funcoes relacioandas a cor que parecem sofrer mais com este problema.
* Terei de retreinar para passar os resultados
* Para selecionar as melhores loss function devemos utilizar outro metodo. Nao sera possivel utilizar as metricas devido a grande quantidade de funcoes que nao gera imagem como a MSE e a SSIM. Essas funcoes tem de ser investigadas. 
* Como o tempo esta curso talvez seja melhor selecionar visualmente as melhores funcoes e agrupar de acordo com as necessidades do script.
* Ainda preciso revisar os excperimentos do sbr

### 3. Entrada no Intervalo [0, 1]:
* Se os dados de entrada da rede estão no intervalo [0,1] e a função de perda gera valores negativos, isso não é problemático por si só. O que importa é como os gradientes resultantes influenciam os pesos.
* Entretanto, é importante que os valores da função de perda sejam coerentes com o intervalo de entrada para manter a estabilidade do treinamento.


Falta ajustar os pesos das funcoes de perda colorchannel principalmente para hsv

In [24]:
#modelos = load_models()
import torch
from loss import *
from models import *
from src.dataload import *
from tqdm.notebook import tqdm
from metrics.metrics import *
loss_battle = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#loss_battle.extend(build_perceptual_losses(rank=device))
#loss_battle.extend(build_channel_losses(rank = device))
loss_battle.extend(build_structural_losses(rank = device))

print(f"{len(loss_battle)} loss functions to train with\n")
#print(f"{len(modelos)} models to train with")

x = torch.randn(1, 3, 256, 256).to(device)
y = torch.randn(1, 3, 256, 256).to(device)
for loss_fn in loss_battle:
    print(f"    {loss_fn.name}, result: {loss_fn(x, y)}     type{type(loss_fn(x, x))})")

4 loss functions to train with

    SSIMLoss, result: 0.49716806411743164     type<class 'torch.Tensor'>)
    PSNRLoss, result: 2.9897866249084473     type<class 'torch.Tensor'>)
    MSELoss, result: 1.9905755519866943     type<class 'torch.Tensor'>)
    GradientLossOpenCV, result: 10.314987182617188     type<class 'torch.Tensor'>)


In [28]:

def normalize_loss_output(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        
        result  = (1/result)
        print(result)

        # Lidar com valores infinitos no tensor
        result = torch.where(torch.isinf(result), torch.tensor(1.0), result)
        result = torch.where(result == -float('inf'), torch.tensor(0.0), result)
        print(result)
        result = result -1
        print(result)
        
        
        # # Obter o valor mínimo e máximo do tensor
        # min_val = torch.min(result)
        # max_val = torch.max(result)
        
        # # Normalização para o intervalo [0, 1]
        # if max_val > min_val:
        #     normalized_result = (result - min_val) / (max_val - min_val)
        # else:
        #     normalized_result = torch.zeros_like(result)
        
        # # Garantir que o tensor esteja no intervalo [0, 1]
        # normalized_result = torch.clamp(normalized_result, 0, 1)
        
        return result
    
    return wrapper

@normalize_loss_output
def a():
    #return torch.Tensor([np.inf])
    return torch.Tensor([9494949])
print(a())

tensor([1.0532e-07])
tensor([1.0532e-07])
tensor([-1.0000])
tensor([-1.0000])


# Outos ajustes ddp

In [ ]:
import os
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.optim as optim
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, DistributedSampler, TensorDataset

def setup(rank, world_size):
    dist.init_process_group("nccl", rank=rank, world_size=world_size)
    torch.cuda.set_device(rank)
    print(f"Rank {rank} initialized.")

def cleanup():
    dist.destroy_process_group()

def train(rank, world_size, epochs=100):
    setup(rank, world_size)

    #dataloader UIEBUIEB
    train_loader_UIEB, test_loader_UIEB,sampler = create_dataloader(dataset_name="UIEB", dataset_path="data",world_size=world_size,rank=rank,rank_test=0)

    model = model.cuda(rank)##carrega o modelo
    ddp_model = DDP(model, device_ids=[rank])

    criterion = nn.MSELoss().cuda(rank)##carrega loss function
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)


    for epoch in range(epochs):
        ddp_model.train()
        sampler.set_epoch(epoch)
        for batch_idx, (data, target) in enumerate(train_loader_UIEB):
            data, target = data.cuda(rank), target.cuda(rank)

            optimizer.zero_grad()
            output = ddp_model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print(f"Rank {rank}, Epoch [{epoch}/{epochs}], Batch [{batch_idx}/{len(dataloader)}], Loss: {loss.item()}")

    cleanup()

if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument("--epochs", type=int, default=10, help="Number of epochs")
    parser.add_argument("--nodes", type=int, default=1, help="Number of nodes")
    parser.add_argument("--gpus", type=int, default=2, help="Number of GPUs per node")
    args = parser.parse_args()

    world_size = args.nodes * args.gpus

    torch.multiprocessing.spawn(train, args=(world_size, args.epochs), nprocs=args.gpus, join=True)


In [ ]:
%torchrun --nnodes=2 --nproc_per_node=2 --node_rank=0 --master_addr="10.228.252.209" --master_port=12355 train_ddp.py --nodes=2 --gpus=2 --epochs=10
#10.228.247.253

In [ ]:
import os
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.optim as optim
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, DistributedSampler, TensorDataset
from loss import *
import torch
from models import *
from src.dataload import *
from tqdm.notebook import tqdm
from metrics.metrics import *

def setup(rank, world_size):
    dist.init_process_group("nccl", rank=rank, world_size=world_size)
    torch.cuda.set_device(rank)
    print(f"Rank {rank} initialized.")

def cleanup():
    dist.destroy_process_group()

class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear = nn.Linear(10, 1)

    def forward(self, x):
        return self.linear(x)

def train_one_model(rank, world_size, epochs, loss_fn, model_name, model):
    setup(rank, world_size)

    #dataloader UIEBUIEB
    train_loader_UIEB, test_loader_UIEB,sampler = create_dataloader(dataset_name="UIEB", dataset_path="data",world_size=world_size,rank=rank,rank_test=0)

    modell = model.cuda(rank)
    ddp_model = DDP(modell, device_ids=[rank])

    criterion = loss_fn().cuda(rank)
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)

    for epoch in range(epochs):
        ddp_model.train()
        sampler.set_epoch(epoch)
        for batch_idx, (data, target) in enumerate(train_loader_UIEB):
            data, target = data.cuda(rank), target.cuda(rank)

            optimizer.zero_grad()
            output = ddp_model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            if batch_idx % 10 == 0:
                print(f"Rank {rank}, Epoch [{epoch}/{epochs}], Batch [{batch_idx}/{len(train_loader_UIEB)}], Loss: {loss.item()}")
    ##Salve Dir para salvar os checkpoints
    ckpt_savedir='output/ckpt_battle/'
    if not os.path.exists(ckpt_savedir):
        os.makedirs(ckpt_savedir)
    if rank == 0:
        # Salvar o estado do modelo original, não o DDP
        torch.save(model.state_dict(), f"{ckpt_savedir}{model_name}_ckpt.pth")
        psnr_list, ssim_list, uciqe_list, uiqm_list = [], [], [], []
        # Avaliar o modelo
        model.eval()
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(test_loader_UIEB):
                data, target = data.cuda(rank), target.cuda(rank)
                #calcula a metrica
                targets = target.cpu().numpy()
                predictions = model(data.cuda(rank)).cpu().numpy()
                psnr_value, ssim_value, uciqe_, uiqm = calculate_metrics(predictions, targets)
                psnr_list.append(psnr_value)
                ssim_list.append(ssim_value)
                uciqe_list.append(uciqe_)
                uiqm_list.append(uiqm)
        avg_ssim = sum(ssim_list) / len(ssim_list)
        avg_psnr = sum(psnr_list) / len(psnr_list)
        avg_uciqe = sum(uciqe_list) / len(uciqe_list)
        avg_uiqm = sum(uiqm_list) / len(uiqm_list)
           
        # Salvar métricas em um arquivo
        results_savedir='output/results_battle/'
        if not os.path.exists(results_savedir):
            os.makedirs(results_savedir)
        
        with open(f'output/{model_name}_metrics.txt', 'w') as f:
            f.write(f"""avg_ssim:{avg_ssim}\navg_psnr:{avg_psnr}\navg_uciqe:{avg_uciqe}\navg_uiqm:{avg_uiqm}""")
            print(f"Metrics for {model_name} saved to {results_savedir}/{model_name}_metrics.txt")


    cleanup()

def train(rank, world_size, epochs):
    
    modelos = load_models()
    loss_battle = []

    loss_battle.extend(build_perceptual_losses())
    loss_battle.extend(build_channel_losses())
    loss_battle.extend(build_structural_losses())

    for model in modelos:
        for loss_fn in loss_battle:
            model_name = model.__class__.__name__ + "_" + loss_fn.__class__.__name__
            train_one_model(rank, world_size, epochs, loss_fn, model_name, model)        
        
if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument("--epochs", type=int, default=100, help="Number of epochs per model")
    parser.add_argument("--nodes", type=int, default=1, help="Number of nodes")
    parser.add_argument("--gpus", type=int, default=2, help="Number of GPUs per node")
    args = parser.parse_args()

    world_size = args.nodes * args.gpus

    torch.multiprocessing.spawn(train, args=(world_size, args.epochs), nprocs=args.gpus, join=True)


# Swim tranformers

In [ ]:
%pip install einops

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from typing import Tuple, List

class PatchEmbedding(nn.Module):
    def __init__(self, in_channels: int = 3, embed_dim: int = 96, patch_size: int = 4):
        super().__init__()
        self.proj = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, x: Tensor) -> Tensor:
        x = self.proj(x)  # [B, C, H, W] -> [B, embed_dim, H', W']
        x = x.flatten(2)  # [B, embed_dim, H', W'] -> [B, embed_dim, H'*W']
        x = x.transpose(1, 2)  # [B, embed_dim, H'*W'] -> [B, H'*W', embed_dim]
        x = self.norm(x)
        return x

class MLP(nn.Module):
    def __init__(self, in_features: int, hidden_features: int = None, out_features: int = None, drop: float = 0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x: Tensor) -> Tensor:
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

class WindowAttention(nn.Module):
    def __init__(self, dim: int, num_heads: int):
        super().__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.scale = (dim // num_heads) ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=False)
        self.proj = nn.Linear(dim, dim)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x: Tensor, mask: Tensor = None) -> Tensor:
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads)
        qkv = qkv.permute(2, 0, 3, 1, 4)  # [3, B, num_heads, N, C // num_heads]
        q, k, v = qkv[0], qkv[1], qkv[2]

        attn = (q @ k.transpose(-2, -1)) * self.scale
        if mask is not None:
            attn = attn.masked_fill(mask == 0, float('-inf'))
        attn = self.softmax(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        return x

class SwinTransformerBlock(nn.Module):
    def __init__(self, dim: int, input_resolution: Tuple[int, int], num_heads: int, window_size: int = 7, shift_size: int = 0, mlp_ratio: float = 4., drop: float = 0.):
        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size

        self.norm1 = nn.LayerNorm(dim)
        self.attn = WindowAttention(dim, num_heads)
        self.drop_path = nn.Identity()

        self.norm2 = nn.LayerNorm(dim)
        self.mlp = MLP(dim, int(dim * mlp_ratio), drop=drop)

    def forward(self, x: Tensor) -> Tensor:
        H, W = self.input_resolution
        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"

        x = x.view(B, H, W, C)

        shortcut = x
        x = self.norm1(x)
        x = x.view(B, H, W, C)

        # Cyclic shift
        if self.shift_size > 0:
            shifted_x = torch.roll(x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
        else:
            shifted_x = x

        x_windows = self.window_partition(shifted_x)
        x_windows = x_windows.view(-1, self.window_size * self.window_size, C)

        attn_windows = self.attn(x_windows)
        attn_windows = attn_windows.view(-1, self.window_size, self.window_size, C)
        shifted_x = self.window_reverse(attn_windows, H, W)

        if self.shift_size > 0:
            x = torch.roll(shifted_x, shifts=(self.shift_size, self.shift_size), dims=(1, 2))
        else:
            x = shifted_x

        x = x.view(B, H * W, C)

        x = shortcut + self.drop_path(x)

        x = x + self.drop_path(self.mlp(self.norm2(x)))

        return x

    def window_partition(self, x: Tensor) -> Tensor:
        B, H, W, C = x.shape
        x = x.view(B, H // self.window


# Rodar no cluster

In [ ]:
torchrun --nnodes=2 --nproc_per_node=2 --node_rank=0 --master_addr="10.228.252.209" --master_port=22 main.py --nodes=2 --gpus=2 --epochs=100


In [ ]:
#local
torchrun --nproc_per_node=2 main.py --nodes=1 --gpus=2 --epochs=100
